# 字符串的定义

* 字符集：有穷的一组字符构成的带有序关系的集合；
* 字符串：就是字符集里有限个元素的排列，可以看做一类特殊的线性表。

# 字符串的相关概念

1. 字符串的长度：字符串中字符的个数，空字符串的长度为0。下面用 |s|表示字符串s的长度；

2. 字符在字符串的位置：字符串中字符顺序排列，从0开始计数，每个字符都有其确定位置或下标；

3. 字符串相等：两个字符串长度相等且相应位置的字符相等；

4. 字典序：字符串之间的一种序关系，简单说就是从左到右开始第一个不相等的字符决定了2个字符串的大小；如果都一致，则长度较短的字符串相对小；

5. 字符串拼接：s1 + s2

6. 子串关系： 称字符串 s1 是字符串 s2 的子串，如果满足： $s_2 = s + s_1 + s^{*}$;即 s1 和 s2 中的一个连续片段一致；
    * 空字符串和本身都是字符串的子串；
    * 子串可以多次出现，且下标可能会重叠，如 babb 相对于 babbabbbbabb.
    
7. 前缀和后缀：
    * 前缀： $s_2 = s_1 + s^{*}$;
    * 后缀： $s_2 = s^{*} + s_1$;
   
8. 串替换： 将字符串中明确下标范围的一段子串进行替换

# 字符串匹配

假设有两个字符串：
    $$t=t_1t_2t_3\dots t_{n-1}$$
    $$p=p_1p_2p_3\dots p_{m-1}$$
字符串匹配就是在字符串t中查找和p相同字符串的过程，下面称t为**目标串**，p为**模式串**，一般情况下 m<<n.

字符串比较的基础是字符比较。

* 如果从字符i开始，目标串的每一个字符都和模式串里的字符一致，则找到了一个匹配的模式串；

* 如果比较过程中，遇到了一对不同的字符，说明目标串里从字符i开始的子串和模式串不匹配。串匹配算法的关键在两点：
    1. 怎样选择开始比较的字符对；
    2. 遇到不匹配的字符对时下一步怎么办
    
    对这两点的不同处理策略，形成了不同的字符串匹配算法。下面介绍朴素匹配算法和KMP匹配算法。

## 朴素匹配算法

最简单的朴素匹配算法，对以上2点的处理策略是：
1. 从左往右依次比较；
2. 发现不匹配，从**当初开始比较位置的下一位置**重新开始匹配

In [9]:
# 只匹配1次
def naive_matching(t,p):
    m,n = len(p),len(t)
    i,j = 0,0
    
    while i<m and j<n:
        if p[i] == t[j]: 
            i,j = i+1, j+1 # 字符匹配就一起前进 
        else:
            i,j = 0, j-(i-1) #如果不匹配，模式串从0开始，目标串从上次比较位置的下一个位置开始 j -(i-1)
    if i == m:
        return j-i
    else:
        return -1

In [10]:
naive_matching(t='assmddmsmcd',p='smc')

7

上面的朴素匹配算法效率很低，原因在于当发现不匹配时，模式串和目标串都要进行回溯。算法的复杂度是：$O(m\times n)$。最糟糕的案例是：
1. t = '00000000000000000000001'
2. p = '001'

效率低的原因在于：
1. 把每次的字符匹配当成完全独立的操作，没有利用字符是有穷的特点，把字符当做随机出现的量；

2. 也没有利用前面匹配过程中的有效信息。

其他改进的算法都是在这个基础上利用字符的特点进行优化的。

## KMP算法(无回溯串匹配算法)

介绍参考：
1. http://www.ruanyifeng.com/blog/2013/05/Knuth%E2%80%93Morris%E2%80%93Pratt_algorithm.html
2. https://www.pythonf.cn/read/107379

In [119]:
def pmt_table(s_str):
    # s模式串
    m,s,pmt = 0,-1,[-1]*len(s_str)

    while m<len(s_str)-1:
        if s == -1 or s_str[m] == s_str[s]:
            m += 1
            s += 1
            pmt[m] = s 
        else:
            s = pmt[s]
    return pmt


def kmp(m,s):
    # m主串，s模式串
    i,j = 0,0 
    pmt = pmt_table(s)
    while i<len(m) and j<len(s):
        # 模式串循环结束，匹配成功；否则失败
        if m[i] == s[j] or j== -1:
            i,j = i+1,j+1
        else:
            j = pmt[j]
    if j == len(s):
        return i-j
    return -1

In [120]:
kmp('ababababc','ababc')

4

kmp算法的复杂度是：$O(m + n)$

# 字符串练习

## [罗马数字转整数](https://leetcode-cn.com/problems/roman-to-integer/submissions/)

In [11]:
class Solution:
    def romanToInt(self, s: str) -> int:
        map_dict={'I':1,'V':5,'X':10,'L':50,'C':100,'D':500,'M':1000,'IV':4,'IX':9,'XL':40,'XC':90,'CD':400,'CM':900}
        ten_binary = 0
        i = 0
        while i < len(s):
            char = s[i]
            slice_char = s[i:(i+2)]  
            if slice_char in map_dict:
                ten_binary += map_dict[slice_char]
                i += 2
            else:
                ten_binary += map_dict[char]
                i += 1
        return ten_binary

In [12]:
s="IX"
a=Solution()
a.romanToInt(s)

9

## [最长公共前缀](https://leetcode-cn.com/problems/longest-common-prefix/submissions/)

In [39]:
class Solution:
    def longestCommonPrefix(self, strs: list) -> str:
        if len(strs) == 0:
            return ''
        prefix,count = strs[0],len(strs)
        
        for i in range(1,count):
            prefix = self.twolcp(prefix,strs[i])
            if prefix == '':
                break 
        return prefix
        
        
    def twolcp(self,str1,str2):
        length,index = min(len(str1),len(str2)),0
        
        while index<length and str1[index] == str2[index]:
            index +=1
        return str1[:index]
    
    def longestCommonPrefix_v2(self, strs: list) -> str:
        if len(strs) == 0:
            return ''
        if len(strs) == 1:
            return strs[0]
        
        s1,strs_1,common_prefix = strs[0],strs[1:],''
        
        bool_v = 1
        for i in range(len(s1)):
            s = s1[i]
            for j in strs_1:
                if len(j)<i+1 or s!=j[i]:
                    bool_v = 0
                    break 
            
            if bool_v:
                common_prefix += s1[i]
            else:
                break
        return common_prefix

In [38]:
strs = ["flower","flow","flight"]
# strs=['aa','a']
a=Solution()
a.longestCommonPrefix(strs)


'fl'

## [有效的括号](https://leetcode-cn.com/problems/valid-parentheses/submissions/)

In [78]:
class Solution:
    def isValid(self, s: str) -> bool:
        if len(s)%2 == 1:
            return False
        
        maps = {')':'(',"}":"{","]":"["}
        left_l = []
        for i in s:
            if i in maps:
                if not left_l or left_l[-1] != maps[i]:
                    return False
                left_l.pop()
            else:
                left_l.append(i)
   
        return len(left_l) == 0
            
                
        
    def isValid_v2(self, s: str) -> bool:
        if len(s)%2 == 1:
            return False
        
        maps = {'(':1,')':-1,'{':2,'}':-2,'[':3,']':-3}
        map_lst = [maps[i] for i in s]
        left_l = []
        for i in map_lst:
            #入栈
            if i>0:
                left_l.append(i)
            else:
                if len(left_l) == 0:
                    return False
                elif i + left_l[-1] != 0:
                    return False
                else:
                    left_l.pop(-1)
        if len(left_l) == 0:
            return True
        else:
            return False
                

In [79]:
s="{}{{[]}}"
a=Solution()
a.isValid(s)

True

## [机器人能否返回原点](https://leetcode-cn.com/problems/robot-return-to-origin/)

In [432]:
class Solution:
    def judgeCircle(self, moves: str) -> bool:
        x,y = 0,0
        for i in moves:
            if i == 'U':
                x += 1
            elif i == 'D':
                x -= 1
            elif i == 'L':
                y += 1
            elif i == 'R':
                y -= 1
        return x == y  == 0
        
    def judgeCircle_v2(self, moves: str) -> bool:
        move_dict = {}
        for i in moves:
            move_dict[i] = move_dict.get(i,0) + 1
            if move_dict[i]>len(moves)/2:
                return False 
        return move_dict.get('U') == move_dict.get('D') and move_dict.get('L') == move_dict.get('R')

In [433]:
s=['UD','LL']
a=Solution()
for i in s:
    print(i,a.judgeCircle(i))

UD True
LL False


## [实现 strStr()](https://leetcode-cn.com/problems/implement-strstr/)

In [121]:
class Solution:
   
    def strStr(self, haystack: str, needle: str) -> int:
        if needle == '':
            return 0
        
        pmt_table = self.pmt(needle)
        i,j = 0,0
        while i<len(haystack) and j<len(needle):
            if j == -1 or haystack[i] == needle[j]:
                i,j = i+1, j+1
            else:
                j = pmt_table[j]
        if j == len(needle):
            return i - j 
       
        return -1
    
    
    def pmt(self,strs):
        pmt_table = [-1]*len(strs)
        m,s = 0,-1
        
        while m<len(strs)-1:
            if s ==  -1 or strs[m] == strs[s]:
                m,s = m+1, s+1
                pmt_table[m] = s
            else:
                s = pmt_table[s]
        return pmt_table

In [122]:
haystack = "hello"
needle = "ll"
a=Solution()
a.strStr(haystack,needle)

2

## [外观数列](https://leetcode-cn.com/problems/count-and-say/)

In [141]:
class Solution:
    def countAndSay(self, n: int) -> str:
        for i in range(1,n+1):
            if i == 1:
                 re = '1'
            else:
                
                a,re  = self.conti_str(re,'')
        return re
    
    def conti_str(self,s,state_s):
        length = len(s)
        if length == 0:
            return '',state_s
        i = 0
        cnt = 1
        while i<length-1 and s[i+1] == s[i]:
                cnt += 1
                i += 1
        state_s +=  str(cnt)+ s[i]
        return self.conti_str(s[(i+1):],state_s)

In [143]:
a=Solution()
a.countAndSay(4)

'1211'

## [最后一个单词的长度](https://leetcode-cn.com/problems/length-of-last-word/)

In [177]:
class Solution:
    def lengthOfLastWord(self, s: str) -> int:
               
        i,empty_bool, res =0,1,0
        while i <len(s):
            char = s[i]
            if char != ' ':
                if empty_bool == 1:
                    res += 1
                else:
                    res = 1
                    empty_bool = 1
            else:
                empty_bool = 0
            i += 1
        return res

In [178]:
s="Hello World"
a=Solution()
a.lengthOfLastWord(s)

5

## [二进制求和](https://leetcode-cn.com/problems/add-binary/)

In [191]:
class Solution:
    def addBinary(self, a: str, b: str) -> str:
        a1,b1 = a[::-1],b[::-1]
        
        i,j,last_div,res = 0,0,0,''
        while i<len(a1) or j<len(b1):
            
            x =int(a1[i])  if i<len(a1) else 0
            y =int(b1[i])  if i<len(b1) else 0
            xy_sum = x + y + last_div
            last_div = xy_sum//2
            res += str(xy_sum%2)
            
            i += 1
            j += 1
        if last_div == 1:
            res += str(last_div)
        return res[::-1] 

In [193]:
a = "11"
b = "1"
t=Solution()
t.addBinary(a,b)

'100'

## [字符串相加](https://leetcode-cn.com/problems/add-strings/submissions/)

## [验证回文串](https://leetcode-cn.com/problems/valid-palindrome/)

In [217]:
class Solution:
    def isPalindrome(self, s: str) -> bool:
        import re 
        s1=''.join([i.lower() for i in re.findall('[a-zA-Z0-9]', s)])
        length = len(s1)
        first_part = s1[:length//2]
        
        if length%2 == 0:
            second_part = s1[length//2:]
        else:
            second_part = s1[(length//2+1):]
        return first_part == second_part[::-1]

In [222]:
s="A man, a plan, a canal: Panama"
t=Solution()
t.isPalindrome(s)

True

## [验证回文字符串 Ⅱ](https://leetcode-cn.com/problems/valid-palindrome-ii/submissions/)

In [471]:
class Solution:
    def validPalindrome(self, s: str) -> bool:
        left,right = 0,len(s)-1
        while left<right:
            if s[left] == s[right]:
                left += 1
                right -= 1
            else:              
                return self.checkPalindrome(s,left+1,right) or self.checkPalindrome(s,left,right-1)
        else:
            return True
        
    def checkPalindrome(self,s,left,right):
        while left<right:
            if s[left] != s[right]:
                return False
            left += 1
            right -= 1
        return True

In [472]:
s=['aba']
t=Solution()
for i in s:
    print(i,t.validPalindrome(i))

aba True


## [反转字符串](https://leetcode-cn.com/problems/reverse-string/submissions/)

In [227]:
class Solution:
    def reverseString(self, s: list) -> None:
        """
        Do not return anything, modify s in-place instead.
        """
        left,right = 0,len(s)-1
        while left<right:
            s[right],s[left]=s[left],s[right]
            left += 1
            right -= 1
        return s

In [229]:
s=["h","e","l","l","o"]
t=Solution()
t.reverseString(s)

['o', 'l', 'l', 'e', 'h']

## [仅仅反转字母](https://leetcode-cn.com/problems/reverse-only-letters/submissions/)

In [701]:
class Solution:
    def reverseOnlyLetters(self, S: str) -> str:
        l,r = 0,len(S)-1
        slst = list(S)
        while l<r:
            while l<r and not S[l].isalpha():
                l += 1
            while l<r and not S[r].isalpha():
                r -= 1 
            slst[r],slst[l] = slst[l],slst[r]
            l += 1
            r -= 1
        return ''.join(slst) 

In [702]:
S="7_28]"
t=Solution()
t.reverseOnlyLetters(S)
# 输出："j-Ih-gfE-dCba"


'7_28]'

## [反转字符串中的元音字母](https://leetcode-cn.com/problems/reverse-vowels-of-a-string/)

In [240]:
class Solution:
    def reverseVowels(self, s: str) -> str:
        vowel = 'aeiouAEIOU'
        s1 = list(s)
        left,right = 0,len(s)-1
        while left<right:
            while left<right and s[left] not in vowel:
                left += 1
            while left<right and s[right] not in vowel:
                right -= 1
                         
            s1[right],s1[left]=s1[left],s1[right]
            left += 1
            right -= 1
        return ''.join(s1)

In [241]:
s='hello'
t=Solution()
t.reverseVowels(s)

'holle'

## [赎金信](https://leetcode-cn.com/problems/ransom-note/)

In [248]:
class Solution:
    def canConstruct(self, ransomNote: str, magazine: str) -> bool:
        mlist = list(magazine)
        i = 0
        while i< len(ransomNote) and ransomNote[i] in mlist:
                mlist.remove(ransomNote[i])
                i += 1
        if i == len(ransomNote):
            return True
        return False

In [250]:
ransomNote = 'aa'
magazine = 'aba'
t=Solution()
t.canConstruct(ransomNote, magazine)

True

## [字符串中的第一个唯一字符](https://leetcode-cn.com/problems/first-unique-character-in-a-string/)

In [268]:
class Solution:
    def firstUniqChar(self, s: str) -> int:
        if len(s) == 0:
            return -1
        slist = list(s)
        unitstr = self.find_str(slist)
        if unitstr == '':
            return -1
        for i in range(len(s)):
            if s[i] == unitstr:
                return i
        
    def find_str(self,slist):
        if len(slist) == 0:
            return ''
        
        char = slist[0]
        slist = slist[1:]
        if char not in slist:
            return char 
        else:
            slist = [i for i in slist if i != char]
            return self.find_str(slist)

In [269]:
s = "aadadaad"
t=Solution()
t.firstUniqChar(s)

-1

## [字符串中的单词数](https://leetcode-cn.com/problems/number-of-segments-in-a-string/submissions/)

In [277]:
class Solution:
    def countSegments(self, s: str) -> int:
               
        i,char_bool, index =0,0,0
        while i <len(s):
            char = s[i]
            if char !=' ':
                if char_bool == 0:
                    index += 1
                    char_bool = 1
            else:
                char_bool = 0
            i += 1
        return index

In [278]:
s = "Hello, my name is John"
t=Solution()
t.countSegments(s)

5

## [反转字符串中的单词 III](https://leetcode-cn.com/problems/reverse-words-in-a-string-iii/)

In [602]:
class Solution:
    def reverseWords(self, s: str) -> str:
               
        i, unit_word,res =0,'',''
        while i <len(s):
            char = s[i]
            if char !=' ':
                unit_word += char
            else:
                res += unit_word[::-1] + char
                unit_word = ''
    
            i += 1
        res += unit_word[::-1]
        return res

In [426]:
s = "Let's take LeetCode contest"
t=Solution()
t.reverseWords(s)

"s'teL ekat edoCteeL tsetnoc"

## [重复的子字符串](https://leetcode-cn.com/problems/repeated-substring-pattern/submissions/)

In [373]:
class Solution:
    def repeatedSubstringPattern(self, s: str) -> bool:
        if len(s)<=1:
            return False
       
        t = s + s
        t1 = t[1:] + t[0]
        m = self.kmp(t1,s)
#         print(t1,s,m)
        return m>=0 and m<len(s)-1
            
    def pmt_table(self,s_str):
        # s模式串
        m,s,pmt = 0,-1,[-1]*len(s_str)

        while m<len(s_str)-1:
            if s == -1 or s_str[m] == s_str[s]:
                m += 1
                s += 1
                pmt[m] = s 
            else:
                s = pmt[s]
        return pmt


    def kmp(self,m,s):
        # m主串，s模式串
        i,j = 0,0 
        pmt = self.pmt_table(s)
        while i<len(m) and j<len(s):
            # 模式串循环结束，匹配成功；否则失败
            if m[i] == s[j] or j== -1:
                i,j = i+1,j+1
            else:
                j = pmt[j]
        if j == len(s):
            return i-j
        return -1

In [374]:
t=Solution()
t.repeatedSubstringPattern("zzz") 

True

## [检测大写字母](https://leetcode-cn.com/problems/detect-capital/submissions/)

In [390]:
# 全部字母都是大写，比如"USA"。
# 单词中所有字母都不是大写，比如"leetcode"。
# 如果单词不只含有一个字母，只有首字母大写， 比如 "Google"。

class Solution:
    def detectCapitalUse(self, word: str) -> bool:
        num = 0
        for i in word:
            if i.isupper():
                num += 1
            
        word_len = len(word)
        return word_len ==  num or num == 0 or (num == 1 and word[0].isupper())

In [391]:
s = ['ABC','abc','Abc','aBC']
t=Solution()
for i in s:
    print(i,t.detectCapitalUse(i)) 

ABC True
abc True
Abc True
aBC False


## [最长特殊序列 Ⅰ](https://leetcode-cn.com/problems/longest-uncommon-subsequence-i/submissions/)

## [反转字符串 II](https://leetcode-cn.com/problems/reverse-string-ii/submissions/)

In [395]:
class Solution:
    def reverseStr(self, s: str, k: int) -> str:
        rest = ''
        return self.rest_reverse(rest,s,k)
        
    def rest_reverse(self,rest,s,k):
        
        if len(s)<k:
            rest += s[::-1]
            return rest
        else:
            part1 = s[:k]
            part2 = s[k:2*k]
            s = s[2*k:]
            rest += part1[::-1] + part2
            return self.rest_reverse(rest,s,k)
    
    
    def reverseStr_v2(self, s, k):
        a = list(s)
        for i in range(0, len(a), 2*k):
            a[i:i+k] = reversed(a[i:i+k])
        return "".join(a)

In [394]:
s = "abcdefg"
k = 2
   
t=Solution()
t.reverseStr(s,k)

'bacdfeg'

## [学生出勤记录 I](https://leetcode-cn.com/problems/student-attendance-record-i/submissions/)

In [412]:
class Solution:
    def checkRecord(self, s: str) -> bool:
        L_list = []
        A_num = 0
        index = 0
        for i in s:
            if i == 'A':
                A_num += 1
            if i ==  'L':
                if len(L_list) == 0 :
                    L_list.append(index)
                else:
                    if index - L_list[-1] == 1:
                        L_list.append(index)
                    else:
                        L_list = [index]
            if A_num>=2 or len(L_list)>=3:
                return False
            
            index += 1

        return A_num<2 and len(L_list)<3
               

In [413]:
s=["PPALLP","PPALLL"]
t=Solution()
for i in s:
    print(i,t.checkRecord(i))

PPALLP True
PPALLL False


## [计数二进制子串](https://leetcode-cn.com/problems/count-binary-substrings/submissions/)

In [523]:
class Solution:
    def countBinarySubstrings(self, s: str) -> int:
        s1,s2 = 0,1
        res = 0
        for i in range(1,len(s)):
            if s[i] == s[i-1]:
                s2 += 1
            else:
                res += min(s1,s2)
                s1 = s2
                s2 = 1
        res += min(s1,s2)
        return res
            
    def countBinarySubstrings_v2(self, s: str) -> int:
        str_map = {'0':'','1':''}
        p = None
        map_dict = {'0':'1','1':'0'}
        cum_dict = {'0':0,'1':0}
        res = []
        for i in s:            
            # cum的计算：入栈
            num_cum = cum_dict[i]
            reverse_cum = cum_dict[map_dict[i]]
            if reverse_cum == 0 or (num_cum != 0 and num_cum<reverse_cum):
                str_map[i] += i
                cum_dict[i] += 1 
            else:
                str_map[i] = i 
                cum_dict[i] = 1
            p = 0 if i == '0' else 1
           
            zero_cum,one_cum = cum_dict['0'],cum_dict['1']
            zero_str,one_str = str_map['0'],str_map['1']
            # 根据cum当前情况抽取符合情况的值
            
            if zero_cum != 0 and one_cum != 0:
                if zero_cum< one_cum:
                    strs = one_str[0:zero_cum] + zero_str
                    
                elif zero_cum > one_cum:
                    strs = zero_str[0:one_cum] + one_str
                else:
                    if p == 1:
                        strs = zero_str[0:one_cum] + one_str
                        cum_dict['0'] = 0
                    else:
                        strs = one_str[0:zero_cum] + zero_str
                        cum_dict['1'] = 0
                        
                        
                res.append(strs)
            
        return len(res)      

In [524]:
s="00110011"
t=Solution()
t.countBinarySubstrings(s)

6

## [转换成小写字母](https://leetcode-cn.com/problems/to-lower-case/submissions/)

## [旋转数字](https://leetcode-cn.com/problems/rotated-digits/submissions/)

In [538]:
class Solution:
    def rotatedDigits(self, N: int) -> int:
        match_str = '2569'
        not_match_str = '347'
        res = 0
        for i in range(2,N+1):
            s = str(i)
            res += all(char not in not_match_str for char in s) and any(char in match_str for char in s)
        return res
                    
                    
    def rotatedDigits_v2(self, N: int) -> int:
        map_dict ={'0':'0','1':'1','8':'8','2':'5','5':'2','6':'9','9':'6'}
        not_match_lst = '347'
        
        def rotatedN(N,num):
            if N<=1:
                return num 
            else:
                s = str(N)
                s1 = ''.join([map_dict.get(i,'') for i in s])
                if len(s1)<len(s) or s1 == s:
                    return rotatedN(N-1,num)
                else:
                    return rotatedN(N-1,num+1)
        
        return rotatedN(N,0)

In [539]:
s=10
t=Solution()
t.rotatedDigits(s)

4

## [唯一摩尔斯密码词](https://leetcode-cn.com/problems/unique-morse-code-words/)

In [553]:
class Solution:
    def uniqueMorseRepresentations(self, words: list) -> int:
        mos = [".-","-...","-.-.","-..",".","..-.","--.","....","..",".---","-.-",".-..","--","-.","---",".--.","--.-",".-.","...","-","..-","...-",".--","-..-","-.--","--.."]
        first = ord('a')
        map_dict = {}
        for i in range(26):
            map_dict.update({chr(first + i):mos[i]})
        s = set()
        for word in words:
            wordmos = ''.join([map_dict[i] for i in word])
            s.add(wordmos)
        return len(s)
    

In [554]:
words = ["gin", "zen", "gig", "msg"]
t=Solution()
t.uniqueMorseRepresentations(words)

2

## [最常见的单词](https://leetcode-cn.com/problems/most-common-word/submissions/)

In [603]:
class Solution:
    def mostCommonWord(self, paragraph: str, banned: list) -> str:
               
        i, unit_word,res =0,'',{}
        while i <len(paragraph):
            char = paragraph[i]
            if char.isalpha():
                unit_word += char.lower()
            elif unit_word:
                if unit_word not in banned:
                    res[unit_word] = res.get(unit_word,0) + 1
                
                unit_word = ''
    
            i += 1
        if unit_word != '':
            res[unit_word] = res.get(unit_word,0) + 1
        max_str,cnt = None,0
        for k in res:
            if res[k]>cnt:
                cnt = res[k]
                max_str = k
                
        return max_str

In [604]:
paragraph = "a, a, a, a, b,b,b,c, c"
banned = ["a"]
t=Solution()
t.mostCommonWord(paragraph,banned)

'b'

## [山羊拉丁文](https://leetcode-cn.com/problems/goat-latin/submissions/)

In [611]:
class Solution:
    def toGoatLatin(self, S: str) -> str:
        i, unit_word,res,index =0,'','',0
        para = 'aeiou'
        while i <len(S):
            char = S[i]
            if char.isalpha():
                unit_word += char
            elif unit_word:
                index += 1
                
                first = unit_word[0].lower()
                if first not in para:
                    unit_word =  unit_word[1:] + unit_word[0] 
                unit_word += 'ma'+ 'a'*index
                
                res += unit_word + char
                
                unit_word = ''
    
            i += 1
        
        index += 1
        first = unit_word[0].lower()
        if first not in para:
            unit_word =  unit_word[1:] + unit_word[0] 
        unit_word += 'ma'+ 'a'*indx

        res += unit_word
        return res   

In [612]:
S="I speak Goat Latin"
t=Solution()
t.toGoatLatin(S)

Latin


'Imaa peaksmaaa oatGmaaaa atinLmaaaa'

## [亲密字符串](https://leetcode-cn.com/problems/buddy-strings/)

In [625]:
class Solution:
    def buddyStrings(self, a: str, b: str) -> bool:
        if len(a) != len(b):
            return False
        if a == b:
            for i in range(len(a)-1):
                if a[i] in a[(i+1):]:
                    return True
            return False
        else:
            alist,blist = list(a),list(b)
            index = 0
            turn_index = 0
            for i in range(len(a)):
                if a[i] != b[i]:
                    index += 1
                    if index == 1:
                        turn_index = i
                    
                    if index == 2:
                        alist[turn_index],alist[i] = alist[i],alist[turn_index]
                        return alist == blist
            return False  

In [624]:
a='abcd'
b='cbad'
t=Solution()
t.buddyStrings(a, b)

True

## [特殊等价字符串组](https://leetcode-cn.com/problems/groups-of-special-equivalent-strings/submissions/)

In [672]:
class Solution:
    from itertools import combinations
    def numSpecialEquivGroups(self, A: list) -> int:
        s = set()
        for word in A:
            s.add(''.join(sorted(word[::2]) + sorted(word[1::2])))
        return len(s)   

In [674]:
A=["abcd","cdab","cbad","xyzz","zzxy","zzyx"]
# A=["abc","acb","bac","bca","cab","cba"]
t=Solution()
t.numSpecialEquivGroups(A)

3

In [ ]:
class Solution:
    def isLongPressedName(self, name: str, typed: str) -> bool:
        

In [ ]:
name = "saeed", typed = "ssaaedd"